# Collecting dynamic information with test cases

You can take information of the current execution frame based on `FrameType` and `codeobject`.

- FrameType: https://docs.python.org/3/reference/datamodel.html#frame-objects
- codeobject: https://docs.python.org/3/reference/datamodel.html#code-objects

In [ ]:
import pytest

In [ ]:
from types import FrameType, TracebackType
from typing import Any, Optional, Callable

import io

output = io.StringIO()

def traceit(frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
#     print(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)
    if frame.f_code.co_filename.endswith("sessions.py"):
        if "Session." in frame.f_code.co_qualname:
        #     print(*args, file=output, **kwargs)
            print(frame.f_lineno, event, frame.f_locals, frame.f_code.co_name, frame.f_code.co_qualname, file=output)
    return traceit

In [ ]:
import sys

sys.settrace(traceit)
pytest.main(["-qqqq", "./cli"])
sys.settrace(None)

contents = output.getvalue()
output.close()

In [ ]:
print(contents)

### Static information again

In [ ]:
path = "./cli/httpie///////////////**/*.py"

from glob import glob

pyfiles = list(glob(path, recursive=True))
len(pyfiles)

In [ ]:
import ast

def getAST(path: str):
    with open(path, "r") as f:
        source = f.read()
        
    return ast.parse(source)

In [ ]:
from ast import NodeVisitor

class NameVisitor(NodeVisitor):
    def __init__(self):
        super().__init__()
        self.names = []
    
    def visit_Name(self, node):
        self.names.append(node.id)
        return super().generic_visit(node)

    
class StaticVisitor(NodeVisitor):
    def __init__(self, focus):
        super().__init__()
        self.linemap = {}
        self.focus_class = focus
        
    def visit_ClassDef(self, node):
        if node.name == self.focus_class:
            for stmt in node.body:
                if not ( isinstance(stmt, ast.FunctionDef) or isinstance(stmt, ast.AsyncFunctionDef) ):
                    nvisitor = NameVisitor()
                    nvisitor.visit(stmt)
                    self.linemap[stmt.lineno] = set(nvisitor.names)
                else:
                    for mstmt in stmt.body:
                        nvisitor = NameVisitor()
                        nvisitor.visit(mstmt)
                        self.linemap[mstmt.lineno] = set(nvisitor.names)
        return super().generic_visit(node)

In [ ]:
myvisitor = StaticVisitor("Session")

for pyfile in pyfiles:
    myast = getAST(pyfile)
    myvisitor.visit(myast)
    
myvisitor.linemap

### How to collect dynamic inforamtion from a project

First, it is necessary define `traceit` function for your purpose.

In [ ]:
from types import FrameType, TracebackType
from typing import Any, Optional, Callable

import json

import pytest
import sys

linemap = {}

def traceit(frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
#     print(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)
    if frame.f_code.co_filename.endswith("sessions.py"):
        if "Session" in frame.f_code.co_qualname:
            """
            varmap = {}
            for var in frame.f_locals:
                varmap[str(var)] = type(var)
            """
            typemap = {}
            for key, value in frame.f_locals.items():
                typemap[key] = type(value)

            linemap[frame.f_lineno] = typemap
            # print(frame.f_lineno, event, frame.f_locals, frame.f_code.co_name, frame.f_code.co_qualname, file=output)
    return traceit

Then, define a `main()` function to run `pytest` or any test cases.

In [ ]:
def main():
    sys.settrace(traceit)
    pytest.main(["-qqqq", "./tests/test_sessions.py"])
    sys.settrace(None)

    for line, locals in linemap.items():
        if line == 154:
            print(line, locals)

if __name__ == "__main__":
    main()